In [1]:
import Bio.SeqIO

In [19]:
import sys

def parse_location(location_str):
    return location_str.split('..')

def parse_string(location_str):
    if location_str.index('complement') == 0:
        return '-'
    else:
        return '+'

def parse_entity(feature):
    '''
    Parse a transcribed entity
    '''
    entity = {}
    # print('feature:', feature)
    
    entity['start'] = feature.location.start.position
    entity['end'] = feature.location.end.position
    entity['strand'] = '+' if feature.location.strand == 1 else '-'
    #print(dir(feature))
    #print("qulaifiers:", feature.qualifiers)
    if 'gene' in feature.qualifiers:
        entity['name'] = feature.qualifiers['gene'][0]
    
    
    for part in feature.location.parts:
        entity['location_parts'] = {
            'start': part.start.position,
            'end': part.end.position,
            'strand': '+' if part.strand == 1 else '-',
        }

    for xref in feature.qualifiers['db_xref']:
        parts = xref.split(':')
        if parts[0] == 'GeneID':
            entity['geneId'] = parts[1]
            break

    if 'transcript_id' in feature.qualifiers:
        entity['transcriptId'] = feature.qualifiers['transcript_id'][0]

    #print('entity:', entity)
    return entity

def extract_refseq_features(refseq_file, sequence_handle=None):
    '''
    Extract a list of the features present in this file. This file should
    only contain sequences for one record.
    
    Parameters
    ----------
    refseq_file: string
        The filename of a refseq file for a single sequence
    sequence_name: string
        The name of the sequence that these refseq annotations are for
    sequence_handle File
        A file handle for the sequence file
        
    Returns
    --------
    features: {}
        A list of features indexed by GenBank gene ids
    '''
    record_count = 0
    genes = {}
    count = 0
    
    for record in Bio.SeqIO.parse(refseq_file, 'genbank'):
        record_count += 1
        #print("dir:", dir(record))
    
        if sequence_handle is not None:
            # store the sequence of this record in the provied fasta
            # file handle
            Bio.SeqIO.write(record, sequence_handle, 'fasta')
        
        #print('record:', record)
        for feature in record.features:
            # print("dir", dir(feature))
            # print(feature.type)
            # print(feature.location, type(feature.location))
            # print(dir(feature))
            try:
                curr_entity = parse_entity(feature)
            except Exception as ex:
                #print("Error parsing feature: {}".format(feature), file=sys.stderr)
                #print("Error: {}".format(ex))
                pass


            curr_entity['type'] = feature.type
            if 'geneId' in curr_entity:
                #print('curr_entity:', curr_entity['geneId'])
                pass

            if feature.type == 'gene':
                curr_entity['mRNAs'] = []
                #print("feature:", feature)
                genes[curr_entity['geneId']] = curr_entity

                if 'pseudo' in feature.qualifiers:
                    curr_entity['type'] = 'pseudo'

                # print("gene", curr_entity)
            elif feature.type == 'CDS':
                #print("feature:", feature)

                cds = {}
                cds['start'] = feature.location.start.position
                cds['end'] = feature.location.end.position
                genes[curr_entity['geneId']]['cds'] = cds
            elif feature.type == 'ncRNA':
                genes[curr_entity['geneId']] = curr_entity
            elif feature.type == 'mRNA':
                if 'mRNAs' not in genes[curr_entity['geneId']]:
                    genes[curr_entity['geneId']]['mRNAs'] = []
                genes[curr_entity['geneId']]['mRNAs'] += [curr_entity]
            elif feature.type == 'misc_RNA':
                genes[curr_entity['geneId']] = curr_entity        
                #print(feature)
            elif feature.type == 'precursor_RNA':
                genes[curr_entity['geneId']] = curr_entity
            elif feature.type == 'tRNA':
                genes[curr_entity['geneId']] = curr_entity
            else:
                # print(feature)
                pass

            count += 1

    print("len:", len(genes))
    return genes

In [25]:
import json
import os
import os.path as op
import urllib

def extract_assembly_annotations(assembly_location):
    #wget -O /tmp/assembly.txt ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/405/GCF_000001405.33_GRCh38.p7/GCF_000001405.33_GRCh38.p7_assembly_report.txt
    !wget -O /tmp/assembly.txt {assembly_location}

    !cat /tmp/assembly.txt | grep "^#" | tail -n 1 > /tmp/assembly_headers.txt
    !cat /tmp/assembly.txt | grep -v "^#" > /tmp/assembly_processed.txt

    with open('/tmp/assembly_headers.txt', 'r') as f:
        headers = f.readlines()[0].strip().split()[1:]
        print(headers)
        
    output_dir = op.join(op.expanduser('~/data/nuccore'),
                         op.splitext(op.basename(assembly_location))[0])

    if not op.exists(output_dir):
        os.makedirs(output_dir)

    output_files = []
    annotations = {
        'sequences': []
    }

    with open('/tmp/assembly_processed.txt', 'r') as f:
        output_fa = op.join(output_dir, 'seq.fa')
        output_chromSizes = op.join(output_dir, 'chromSizes.tsv')
        
        if op.exists(output_fa):
            os.remove(output_fa)
        
        if op.exists(output_chromSizes):
            os.remove(output_chromSizes)
            
        f_chromsizes = open(op.join(output_dir, 'chromSizes.tsv'), 'a')
            
        for line in f:
            parts = line.strip().split()
            #print("parts:", parts)
            genbank_accn = parts[6]
            sequence_name = parts[0]
            ucsc_name = parts[-1]
            refseq_name = parts[-5]
            seq_len = int(parts[-2])
            
            common_name = ucsc_name if ucsc_name != 'na' else refseq_name
            
            f_chromsizes.write("{}\t{}\n".format(common_name, seq_len))
            #print('common_name:', common_name)

            output_file  = op.join(output_dir, '{}.gb'.format(genbank_accn))
            done_file = '{}.done'.format(output_file)
            #print(output_file)

            if not op.exists(done_file):
                try:
                    download_file = output_file + '.orig'
                    url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nuccore&id={}&rettype=gbwithparts&retmode=text'.format(genbank_accn)
                    #print("url:", url)
                    urllib.request.urlretrieve(url, output_file)
                    with open(done_file, 'w') as f1:
                        pass
                except Exception as ex:
                    print('Ex:', ex)
                    continue

            #output_struct = dict(zip(headers, parts))

            output_struct = {}
            output_struct['refseqFilename'] = output_file
            output_struct['sequence_name'] = sequence_name
            output_struct['ucsc_name'] = ucsc_name
            output_struct['refseq_name'] = refseq_name
            output_struct['gene_features'] = extract_refseq_features(output_file,
                                                                     sequence_handle=output_fa)
            output_files += [output_struct]
            annotations['sequences'] += [output_struct]
        print("len(annotations)", len(annotations['sequences']))

    with open(op.join(output_dir, 'annotations.json'), 'w') as fo:
        json.dump(annotations, fo, indent=2)
    #print(output_files)
    return output_dir

In [26]:
#
# assembly_location="ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/215/GCF_000001215.4_Release_6_plus_ISO1_MT/GCF_000001215.4_Release_6_plus_ISO1_MT_assembly_report.txt"
# 
# assembly_location="ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/405/GCF_000001405.37_GRCh38.p11/GCF_000001405.37_GRCh38.p11_assembly_report.txt"

# h37rv
# assembly_location="ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/195/955/GCF_000195955.2_ASM19595v2/GCF_000195955.2_ASM19595v2_assembly_report.txt"

# sacCer3
assembly_location='ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/146/045/GCF_000146045.2_R64/GCF_000146045.2_R64_assembly_report.txt'

output_dir = extract_assembly_annotations(assembly_location)


--2018-04-30 22:56:35--  ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/146/045/GCF_000146045.2_R64/GCF_000146045.2_R64_assembly_report.txt
           => ‘/tmp/assembly.txt’
Resolving ftp.ncbi.nlm.nih.gov... 2607:f220:41e:250::12, 130.14.250.12
Connecting to ftp.ncbi.nlm.nih.gov|2607:f220:41e:250::12|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /genomes/all/GCF/000/146/045/GCF_000146045.2_R64 ... done.
==> SIZE GCF_000146045.2_R64_assembly_report.txt ... 2713
==> EPSV ... done.    ==> RETR GCF_000146045.2_R64_assembly_report.txt ... done.
Length: 2713 (2.6K) (unauthoritative)

GCF_000146045.2_R64 100%[===================>]   2.65K  --.-KB/s    in 0s      

2018-04-30 22:56:36 (6.53 MB/s) - ‘/tmp/assembly.txt’ saved [2713]

['Sequence-Name', 'Sequence-Role', 'Assigned-Molecule', 'Assigned-Molecule-Location/Type', 'GenBank-Accn', 'Relationship', 'RefSeq-Accn', 'Assembly-Unit', 'Sequence-Length', 'UCSC-s

In [24]:
def annotations_to_lines(annotations_json_file):
    '''
    Convert a list of annotations in JSON format to a text file.
    
    Parameters
    ----------
    annotations_json_file: string
        The filename containing the json file with the gene annotations
    
    Returns
    -------
        A list of lines containing refGene formatted lines with the annotations
    '''
    with open(annotations_json_file, 'r') as f:
        json_file = json.load(f)
    
        for sequence in json_file['sequences']:
            common_name = sequence['ucsc_name'] if sequence['ucsc_name'] != 'na' else sequence['refseq_name']
            print('common_name', common_name)
            
            for gene in sequence['gene_features'].values():
                if 'cds' in gene:
                    print("{chr}\t{start}\t{end}".format(
                        chr=common_name,
                        start=gene['cds']['start'],
                        end=gene['cds']['end']))
                else:
                    print
                
            

In [6]:
lines = annotations_to_lines(op.join(output_dir, 'annotations.json'))

common_name chrM
chrM	13817	26701
chrM	13817	23167
chrM	13817	21935
chrM	13817	19996
chrM	13817	18830
chrM	13817	16322
chrM	24155	25255
chrM	27665	27812
chrM	28486	29266
chrM	36539	43647
chrM	36539	42251
chrM	36539	40265
chrM	36539	38579
chrM	46722	46953
chrM	48900	50097
chrM	61021	61729
chrM	73757	74513
chrM	74494	75984
chrM	79212	80022


In [7]:
output_dir

'/Users/pete/data/nuccore/GCF_000146045.2_R64_assembly_report'